In [1]:
"""
les symboles particulières comme ? ! et . sont supprimé dans la question.
Il faut peut être returner une réponse spécifique si il rencontre plus 1 mot qu'il existe pas dans le corpus
Nous suprimmons également:
  1. Mot qui contient une seule lettre(qui sont suivante des erreurs)
  2. Une fonction unique pour nettoyage tous les données dans train-test et chat
Nous allons traiter tous les mots clés suivant :
km, vitesse, carburant, huile, position, batterie, pression_pneu. Il reste trajet et vin
  
"""
#sys
#!{sys.executable} -m pip uninstall gtts

#from gtts import gTTS
#from playsound import playsound
#import speech_recognition
from __future__ import unicode_literals, print_function, division
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french", ignore_stopwords = False)
import csv
import re
from io import open
import unicodedata
import string
import re
import random
import os
import math
import time
from dateparser.search import search_dates
import pandas as pd
import data_processed
import config


CONVOS_TRAIN, QUESTION, CONVOS_TEST, QUESTION_TEST= data_processed.data_processed()
CONVOS_TRAIN.extend(CONVOS_TEST)
DATA = data_processed.convos_to_questions(CONVOS_TRAIN)
#start = time.time()
#DICTIONARY =  data_processed.dict_for_all_doc(DATA)
KEY_LIST_  = data_processed.get_key_list(CONVOS_TRAIN)
COMPLET_DICT_TIME = data_processed.date_time_()
CITY_LIST = data_processed.construct_list_city()
LIST_OF_WORDS = data_processed.list_words(DATA)
K  = 1/10
STOPWORDS = stopwords.words('french')


def clearing_word(word):
    word = re.sub('\x8e', 'é', word)
    word = re.sub('\x88', 'à', word)
    word = re.sub('\x9d', 'ù', word)
    word = re.sub('\x8f', 'è', word)
    word = re.sub('\x9e', 'û', word)
    word = re.sub('\x90', 'ê', word)
    word = re.sub('\x99', 'ô', word)
    word = re.sub('\x94', 'î', word)
   # word = re.sub('\x8f', 'è', word)
    word = re.sub('\x8d', 'ç', word)
    word = re.sub('õ', '', word)
    word = re.sub('Ê', '', word)
    word = re.sub('[?,.,!, \,,  %]', '', word)
    if word == 'û' or word == 'v' or word == 'é':
        word = ''
    if word =="2017êles":
        word = "2017"
    if "ênox" in word:
        word ="nox"
    return word


def clear_line(pharse):
    """
    Arg: string
    Return: string
    """
    dict_manuel = dict_manuelle()
    pharse_ =[]
    for words in pharse.strip().strip('?').strip('.').strip('!').lower().split(' '):
        for word in words.split('-'):
            for word_ in word.split('\''):
                word_ = clearing_word(word_)
                word_ = stemmer.stem(word_)
        #if word_ not in stop_word and len(word_) >1:       
        pharse_.append(word_)  
        for i in range(len(pharse_)):
            for line in dict_manuel:
                if pharse_[i] in line:
                    pharse_[i] = line[0]
    return ' '.join(pharse_)

def stemmer_line(line):
    return ' '.join([stemmer.stem(word) for word in line.split()])

def prepareData(PAIRS):
    pairs_trains, pairs_tests = [], []
   # PAIRS.extend(pairs1)
    index_train = random.sample(range(len(PAIRS)), int(len(PAIRS)*0.80))
    for i in range(len(PAIRS)):
        if i in index_train:
            pairs_trains.append(PAIRS[i])
        else:
            pairs_tests.append(PAIRS[i])
    print("Read %s sentence pairs of training set" % len(pairs_trains))
    print("Read %s sentence pairs of test set" % len(pairs_tests))
    return pairs_trains, pairs_tests

def get_all_convos(stopwords, file):
    
    convos = []
    questions_to_print = []
    liste_file = [file]
    list_of_word = []
    for file in liste_file:
        with open(file) as f:
            i=0
            for line in f:
                if i%2==0:
                    question = str(line)
                    if '++++' in question:
                         question = question[8:]
                    question_ = question.strip().strip('!').strip('?').strip('.')
                    questions_to_print.append(question_)
                    question = clear_line(question)
                else:
                    answer = str(line)
                    if '++++' in answer:
                        answer = answer[8:].strip()
                    convos.append([question, answer])
                i+=1
        f.close()
        good_index = []
        convos_ = []
        for i in range(len(convos)):
            if convos[i][0] not in convos_:
                convos_.append(convos[i][0])
                good_index.append(i)
            
    
    return [convos[i] for i in good_index], [questions_to_print[i] for i in good_index]


def get_questions_for_classification():
    
    file1 = 'data/trains_sujet.txt'
    file2 = 'data/trains_hors_sujet.txt'
    liste_file = [file1, file2]
    index = 0
    for file in liste_file:
        questions = []
        with open(file) as f:
            i = 0 
            for line in f:
                if i%2 == 0:
                    question = str(line).strip().strip('?').strip('.').strip('!')
                    if '++++' in question:
                         question = question[8:]
                    questions.append(question)
                i += 1
        if index == 0:
            questions_sujet = questions
            index += 1
        else:
            questions_hors = questions
        f.close()
    return questions_sujet, questions_hors


def get_all_questions_to_print(file):
    
    convos = []
    liste_file = [file]
    list_of_word = []
    for file in liste_file:
        with open(file) as f:
            i=0
            for line in f:
                if i%2==0:
                    question = str(line)
                    if '++++' in question:
                         question = question[8:]
                    question = question.strip()
                    convos.append(question)
                i+=1
        f.close()
        indexes = []
        index = 0
    for question in convos:
        question = clear_line(question)
        if len(question) !=0:
                indexes.append(index)
        index +=1
    return [convos[i] for i in indexes]

def get_list_words(convos):
    list_words =[]
    for pair in convos:
        for word in pair[0].split():
            if word not in list_words:
                list_words.append(word)
    print('There are {} words in corpus:'.format(len(list_words)))
    return list_words
                

def simulation_pairs():
    
    DIRECT_VARIABLE = ['vitesse', 'batterie', 'position', 'km']
    BORDE = {'MAX': ['plus grand', 'plus grande', 'plus grands', 
         'plus grandes',
         'plus vite', 'max', 'maximum', 'maximal', 'maximaux', 'maximale',
        'plus haut', 'plus haute', 'plus hautes', 'plus hauts',  
        'plus élevé', 'plus élevée', 'plus élevés', 'plus élevées'], 
         'MIN':['moins grand', 'moins grande', 'moins grands','moins grandes',
        'plus petit', 'plus petite', 'plus petits', 'plus petites','plus faible', 'plus faibles', 
         'min', 'minimal', 'minimale', 'minimales', 'minimaux','moins vite', 
         'moins élevé', 'moins élevée', 'moins élevés', 'moins élevées'],
        'MOYENNE': ['moyenne', 'moyen', 'moyennement']
        }
    COMPLEX_ANALYSIS = ['problème', 'problèmes', 'erreurs', 'erreur', 'danger']
    PAIRS = []
    question_for_simulation = ['quels véhicules ont', 'quel véhicule', 'quelle voiture', 'quelles voitures']
    reponse_for_simulation = 'véhicule'
    for borde in BORDE:
        for word in BORDE[borde]:
            for direct_variable in DIRECT_VARIABLE:
                for question_f_s in question_for_simulation:
                    if direct_variable !='position':
                        question= question_f_s+' '+direct_variable + ' '+ word
                        key0 = '#id#'
                        key1 = '#'+direct_variable+'#'
                        key2 = '#'+borde +'#'
                        reponse = reponse_for_simulation +' '+key0+ ' '+ key1+ ' '+ key2
                        PAIRS.append([question,reponse])

    question_for_simulation = ['', 'quel véhicule a', 'quelle voiture a', 'quelles voitures ont', 'il y a']
    reponse_for_simulation = 'véhicule' 
    for direct_variable in DIRECT_VARIABLE:
        for complex_analysis in COMPLEX_ANALYSIS:
            for question_f_s in question_for_simulation:
                #if direct_variable !='position':
                      question= question_f_s+' '+complex_analysis + ' '+ direct_variable
                      key0 = '#id#'
                      key1 = '#'+direct_variable+'#'
                      key2 = '#'+ complex_analysis +'#'
                      reponse = reponse_for_simulation +' '+key0+ ' '+ key2 +' '+ key1
                      PAIRS.append([question,reponse])
    return PAIRS

def clear_pairs_trains(pairs_trains):
    questions = []
    index = []
    for i in range(len(pairs_trains)):
        if  pairs_trains[i][0] not in questions:
            index.append(i)
            questions.append(pairs_trains[i][0])
    return [pairs_trains[i] for i in index]

def tf_idf(word, doc):
    """
    Nous avons utilisé une normalisation pour TF-IDF"""
    tf, df = 0, 0
    len_doc = len(split_words(doc))
    value_word = 1/math.sqrt(len_doc)
    value_word = math.sqrt(value_word)
    if word in str(doc):
        tf = value_word
    for doc_ in DATA:
        if word in str(doc_):
            df+=1
    if df>0:
        return tf*math.log(len(DATA)/(df), 10)
    else:
        return 0
    
def tf_idf_modified(word, doc, k):
    """
    Nous avons utilisé une normalisation pour TF-IDF"""
    tf, df = 0, 0
    len_doc = len(split_words(doc))
    value_word = 1/math.sqrt(len_doc)
    value_word = math.sqrt(value_word)
    len_word = len(word.split())
    #len_word *= len_word
    if word in str(doc):
        tf = value_word*k/len_word
    for doc_ in DATA:
        if word in str(doc_):
            df+=1
    if df>0:
        return tf*math.log(len(DATA)/(df), 10)
    else:
        return 0

def dict_for_all_doc(data): 
    """
    return hold data's dictionary"""
    dict_ = {}
    for doc in data:
        list_word_ = split_words(doc)    
        dict_[doc] = {word: tf_idf_modified(word, doc, K) for word in list_word_}
    return dict_
    
def dict_doc_score_for_classif(doc, sujet): 
    """
    We return score of all one words and all two consequent words"""

    list_word_ = split_words(doc)    
    return {word: tf_idf_for_classif(word, doc, sujet) for word in list_word_}

   
def split_words(doc):
    """
    Return word and term, for example, if document of 5 words, 
    it will return 5+4+3 words and terms"""
    list1 = doc.split()
    list2 = []
    for word in list1:
        list2.append(word)
    for i in range(len(list1)-1):
        word = list1[i]+ ' '+list1[i+1]
        list2.append(word)
        for i in range(len(list1)-2):
            word = list1[i]+' '+list1[i+1]+list1[i+2]
            list2.append(word)
    return list2
        
        
def get_key_list(pairs_trains):
    
    dict_ = dict_manuelle()
    KEY_LIST = []
    KEY_LIST_ = []
    for pair in pairs_trains:
        for word in pair[1].split():
            if '#' in word:
                word = re.sub('#', '', word)
                if word not in KEY_LIST:
                    KEY_LIST.append(word)
    for key in KEY_LIST:
        key = stemmer.stem(key)
        if key not in KEY_LIST_ and '_' not in key:
            KEY_LIST_.append(key)
    KEY_RETURN = []
    for key in KEY_LIST_:
        boolean = False
        for list_ in dict_:
            if key in list_:
                KEY_RETURN.append(list_[0])
                boolean = True
                break
        if not boolean:
            KEY_RETURN.append(key)
    #for list_ in dict_:
       # if list_[0] not in KEY_RETURN:
        #    KEY_RETURN.append(list_[0])       
    return KEY_RETURN
        

def find_right_index(new_doc,DICT, méthode = 'normal'):
    
    index=0
    biggest_score = score_by_new_doc(new_doc, QUESTION_TO_TRAINS[0],DICT, méthode)
    for i in range(len(QUESTION_TO_TRAINS)):
        boolean1 = score_by_new_doc(new_doc, QUESTION_TO_TRAINS[i], DICT, méthode) >biggest_score
        boolean2 = score_by_new_doc(new_doc, QUESTION_TO_TRAINS[i], DICT, méthode) == biggest_score
        boolean3 = len(QUESTION_TO_TRAINS[index]) >len(QUESTION_TO_TRAINS[i])
        if boolean1 or (boolean2 and boolean3):
            biggest_score = score_by_new_doc(new_doc, QUESTION_TO_TRAINS[i],DICT, méthode)
            index = i
    précision, rappel = precision_recall(new_doc, QUESTION_TO_TRAINS[index])
    #if biggest_score ==0 or précision <0.2 or rappel<0.2:
     #   for question in QUESTION_TO_TRAINS:
      #      if '_return_' in question:
       #         try:
        #            index = QUESTION_TO_TRAINS.index(question)
         #       except ValueError:
          #          index = -1
    return index

def reformule_question_if_need(new_question, good_question):
    
    if 'to_return' in good_question:
        return True
    score1 = score_by_new_doc(new_question, good_question, DICT)
    score2 = 0
    for word in Dict_of_methode[good_question]:
        score2 += Dict_of_methode[good_question][word]
    if score1/score2 > 0.6:
        return  True
    else:
        return  False


def test(DICT, méthode, train =False):
    
    if not train:
        index_to_print = random.sample(range(len(pairs_tests)), 20)
        #loss_total = 0
        for i in range(len(pairs_tests)):
            index = find_right_index(pairs_tests[i][0], DICT, méthode)
            #loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_tests[i][1])
            #loss_total +=loss
            if i in index_to_print:
                print(pairs_tests[i])
                print(pairs_trains[index][1])
        print('-'*80)
       # print('ACCURACY=', 1-loss_total/len(pairs_tests))
    if  train:
        #loss_total = 0
        #index_to_print = random.sample(range(len(pairs_trains)), 20)  
        list_of_bad_prediction = []
        i0 = 0
        for i in range(len(pairs_trains)):
            index = find_right_index(pairs_trains[i][0], DICT, méthode)
            #loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_trains[i][1])
            #loss_total +=loss
            if index != i :
                i0 +=1
                print(pairs_trains[i])
                print(pairs_trains[index][1])
        print('-'*80)
        #print('ACCURACY=', 1-loss_total/len(pairs_trains))
        print('Il y a {} erreurs d\'indexes parmis {} prédictions'.format(i0, len(pairs_trains)))

        
def test_without_print(DICT, méthode, train =False):
    
    if not train:
        loss_total = 0
        for i in range(len(pairs_tests)):
            index = find_right_index(pairs_tests[i][0], DICT, méthode)
            loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_tests[i][1])
            loss_total +=loss
        print('-'*80)
        print('ACCURACY=', 1-loss_total/len(pairs_tests))
    if  train:
        loss_total = 0
        index_to_print = random.sample(range(len(pairs_trains)), 20)   
        for i in range(len(pairs_trains)):
            index = find_right_index(pairs_trains[i][0], DICT, méthode)
            loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_trains[i][1])
            loss_total +=loss
        print('-'*80)
        print('ACCURACY=', 1-loss_total/len(pairs_trains))

def change_subjet_of_question(question):
    dict_ = {'ton':'mon', 'ta':'ma', 'votre':'mon', 'tes':'mes', 'mon':'ton', \
            'ma':'ta', 'mes':'tes', 'tu': 'je', 'moi':'toi', 'je':'tu', 'toi':'moi',\
            'vous':'je'}
    question_ = []
    for word in question.lower().split():
        if word in dict_:
            word = dict_[word]
        question_.append(word)
    return ' '.join(question_)
        
def chat(méthode='normal'):
    
    print('Bonjour, C\'est le bot d\'Avicen, pose tes questions, s\'il te plaît!')
    path = 'processed/chat_tfidf.txt'
    f =  open(path, 'a+') 
    while True:
            line = str(input('Vous: '))
            list_city = _find_right_geography(line, CITY_LIST)
            line = line.lower().strip().strip('?').strip('.')
            time, time_word = _find_right_time(line, COMPLET_DICT_TIME)
            if time_word is not None:
                line = line.replace(time_word, '')
            if list_city is not None:
                for city in list_city:
                    line = line.replace(city, '')
            LINE = clear_line(line)
            unknown_word =0
            for word in LINE.split():
                if word not in LIST_OF_WORDS:
                    unknown_word += 1
            #if unknown_word > 1:
             #   print('Désolé, je ne comprends pas ta question, peux tu la reformuler s\'il te plaît?')
              #  continue
            if len(line) !=0:
                index= find_right_index(LINE, DICT, méthode)
                good_question = QUESTION_TO_TRAINS[index]
                if not reformule_question_if_need(LINE, good_question):
                    get_answer = str(input('Tu veux demander: {}?  |'.format(questions_sujet[index])))
                    if 'non' in get_answer.lower():
                        print('Peux tu reformuler ta question s\'il te plaît?')
                        continue
                if time == None and list_city == None:
                     print('bot: ', pairs_trains[index][1])
                elif list_city ==None:
                    print('Bot: {}, {}'.format(time_word, pairs_trains[index][1]))
                elif time == None:
                    cities = ' '.join(list_city)
                    print('Bot: À {}, {}'.format(cities, pairs_trains[index][1]))
                else:
                    cities = ' '.join(list_city)
                    print('Bot: À {}, {}, {}'.format(cities, time_word, pairs_trains[index][1]))
                f.write('VOUS ++++ '+line+'\n')
                f.write('BOT ++++ '+pairs_trains[index][1]+'\n')
            
            else:
                f.close()
                break 

def precision_recall(lstcomp, lstref):
    
    card_intersec = 0.0 # force à utiliser la division non entière
    for t in set(lstcomp) :
        card_intersec += min(lstref.count(t), lstcomp.count(t))
    if len(lstcomp)==0:
        precision =1
    else:
        precision = card_intersec/len(lstcomp)
    if len(lstref)==0:
        rappel = 1
    else:
        rappel = card_intersec/len(lstref)
    return (precision, rappel)


def calcul_precision_recall_(train = False): 
    
    if not train:
        total_precision, total_recall = 0, 0
        len_ = len(convos_test)
        index_to_print = random.sample(range(len_), 30)
        for i in range(len(questions_test)):
            question = convos_test[i][0]
            index = find_right_index_(question)
            answer = convos_train[index][1]
            good_answer = convos_test[i][1]
            precision, recall = precision_recall\
            (answer, good_answer)
            total_precision  += precision
            total_recall     += recall
            #if i in index_to_print:
            #    print('raw questions: {}. precision {}'.format( questions_test[i], precision))
             #   print('answer: {}. good answer: {} '.format(answer, good_answer))
        #print('-'*80)
        total_precision = total_precision/len_
        total_recall = total_recall/len_      
        F = 2*total_precision*total_recall/(total_precision+total_recall)
        print('ON TEST, WE HAVE: PRECISION = {:.5f},  RECALL = {:.5f}, F_MESURE = {:.3f} '\
              .format(total_precision, total_recall,F))
        print()
    if  train:       
        total_precision, total_recall = 0, 0
        len_total = len(convos_train)
        for i in range(len(convos_train)):
            question = convos_train[i][0]
            index = find_right_index_(question)
            answer = convos_train[index][1]
            good_answer = convos_train[i][1]
            precision, recall = precision_recall\
            (answer, good_answer)
            total_precision  += precision
            total_recall     += recall
        print('-'*80)
        total_precision = total_precision/len_total
        total_recall = total_recall/len_total     
        F = 2*total_precision*total_recall/(total_precision+total_recall)
        print('ON TRAINS, WE HAVE PRECISION = {:.5f},  RECALL = {:.5f}, F_MESURE = {:.3f} '\
              .format(total_precision, total_recall,F))
        

def return_F_score(): 
    
        total_precision, total_recall = 0, 0
        len_ = len(convos_test)
        for i in range(len(questions_test)):
            question = convos_test[i][0]
            index = find_right_index_(question)
            answer = convos_train[index][1]
            good_answer = convos_test[i][1]
            precision, recall = precision_recall\
            (answer, good_answer)
            total_precision  += precision
            total_recall     += recall
        total_precision = total_precision/len_
        total_recall = total_recall/len_     
        F = 2*total_precision*total_recall/(total_precision+total_recall)
        return F
 
    
def chatting_with_corpus():

    path = 'processed/chat_tfidf.txt'
    i = 0
    questions = []
    with open(path, 'r') as f:
        for line in f:
            if i%2 ==0:
                line = line[8:]
                line = line.strip().strip('-').strip('+')
                questions.append(line)
            i +=1
    f.close()
    good_index = []
    questions_ = []
    index = 0
    for question in questions:
        question = clear_line(question)
        if question not in questions_:
            questions_.append(question)
            good_index.append(index)
        index += 1
    questions_clean = [questions[i] for i in good_index]
    for line in questions_clean:
            print('Humaine: ', line)
            time, time_word = _find_right_time(line, COMPLET_DICT_TIME)
            list_city = _find_right_geography(line, CITY_LIST)
            if time_word is not None:
                line = line.replace(time_word, '')
            if list_city is not None:
                for city in list_city:
                    line = line.replace(city, '')
            LINE = clear_line(line)
            unknown_word =0
            for word in LINE.split():
                if word not in LIST_OF_WORDS:
                    unknown_word += 1
            if unknown_word > 1:
                print('Désolé, je ne comprends pas ta question, peux tu la reformuler s\'il te plaît?')                    
                print()
                continue
            if len(line) !=0:
                index= find_right_index_(LINE)
                if time == None and list_city == None:
                     print('Bot: ', convos_train[index][1])
                elif list_city ==None:
                    print('{}, {}'.format(time_word, convos_train[index][1]))
                elif time == None:
                    cities = ' '.join(list_city)
                    print('À {}, {}'.format(cities, convos_train[index][1]))
                else:
                    cities = ' '.join(list_city)
                    print('À {}, {}, {}'.format(cities, time_word, convos_train[index][1]))
            print()
               
def dict_manuelle():
    """
    return la liste des listes des synonymes"""
    list_ = []
    dict_ = {}
    dict_['voiture'] = ['voiture', 'véhicule']
    dict_['km'] = ['km', 'kilomètrage', 'kilométrage', 'distance', 'kilomètre']
    dict_['essence'] = ['essence', 'carburant', 'énergie']
    dict_['roule'] = ['roule', 'roulent', 'marche', 'marcher', 'circule', 'circulent', 'circulation']
    dict_['bien'] = ['bien', 'bonne', 'good', 'excellent', 'parfait', 'beau', 'ok']
    dict_['min']  = ['min', 'minimum', 'minimal']
    dict_['pneu'] = ['pneumatique', 'pneu', 'pneus']
    dict_['pouquoi'] = ['cause', 'pourquoi', 'raison', 'motif']
    dict_['stupid'] = ['stupid', 'con', 'idiot', 'bête', 'imbécile']
    dict_['connaître'] = ['connaître', 'connaitre', 'savoir', 'comprendre',\
                          'connaisance', 'connais', 'connaîs', 'sais', 'savez']
    dict_['problème']  = ['problème', 'difficulté', 'danger', 'ennui', 'dangers']
    dict_['erreur'] = ['faute', 'erreur']
    dict_['arrêt'] = ['arrêt', 'arrêter', 'stopper', 'immobilisé', 'immobiliser', 'immobile', 'paralisé']
    dict_['disponible'] = ['disponible', 'disponibilité', 'libre']
    dict_['abimer'] = ['abîmer', 'hors service', 'cassé', 'endommager', 'abîmé']
    dict_['boitier'] = ['boitier', 'boîtier', 'boîte']
    dict_['frein'] = ['frein', 'freinage', 'freiner']
    dict_['parcouru'] = ['parcouru', 'parcourus', 'parcourir']
    dict_['peux'] = ['peux', 'pouvoir', 'pourrais']
    dict_['veux'] = ['veux', 'vouloir', 'veut', 'voulais', 'voudrais', 'souhaiter', 'souhaite']
    dict_['fait'] = ['fait', 'fais', 'faire', 'faites']
    dict_['fort'] = ['fort', 'puissant', 'robuste', 'solide']
    dict_['grand'] = ['grand', 'gross', 'élevé', 'haut', 'haute']
    dict_['mal'] = ['mal', 'mauvais', 'souci']
    dict_['changer'] = ['modifier', 'modification', 'changer', 'changement']
    dict_['avoir'] = ['as', 'obtenir', 'avoir', 'posséder', 'disposer']
    dict_['peur']  = ['peur', 'craint']
    dict_['placer'] = ['placer','installer']
    dict_['nombre'] = ['nombre', 'combien', 'quantité']
    dict_['aider']  = ['aider', 'aide', 'soutien', 'soutenir']
    dict_['question'] = ['question', 'demande']
    dict_['information'] = ['information', 'infos', 'renseignement', 'indictation',\
                            'informer', 'renseigner','indiquer']
    dict_['usé'] = ['usé', 'usure', 'vieux', 'vieille', 'fatigué']
    dict_['réduire']  = ['réduire', 'réduction', 'abaisser', 'diminuer', 'diminution']
    dict_['présent']  = ['présent', 'maitenant', 'actuel', 'actuellement']
    dict_['suivre']   = ['suivre', 'observer', 'surveiller', 'poursuivre']
    dict_['aller']    = ['aller', 'vas', 'vais', 'va']
    dict_['échanger'] = ['échanger', 'estimer', 'échangement', 'estimation',\
                         'évaluer', 'calculer','évaluation'\
                        , 'mesurer']
    dict_['taux']    = ['taux', 'pourcentage']
    dict_['ampoule'] = ['ampoule', 'feux', 'feu']
    for key in dict_:
        l = []
        for word in dict_[key]:
            l.append(stemmer.stem(word))
        list_.append(l)
    return list_

    
                
def chatting_with_test_corpus():
    
    path = 'test.txt'
    questions = []
    with open(path, 'r') as f:
        for line in f:
                line = line.strip()
                questions.append(line)
    f.close()
    questions_clean = []
    for question in questions:
        if question not in questions_clean:
            questions_clean.append(question)
    for line in questions_clean:
            line1 = line
            time, time_word = _find_right_time(line, COMPLET_DICT_TIME)
            list_city = _find_right_geography(line, CITY_LIST)
            if time_word is not None:
                line = line.replace(time_word, '')
            if list_city is not None:
                for city in list_city:
                    line = line.replace(city, '')
            LINE = clear_line(line)
            unknown_word =0
            print('Vous: ', line1)
            for word in LINE.split():
                if word not in LIST_OF_WORDS:
                    unknown_word += 1
            if unknown_word > 1:
                print('Bot: Désolé, je ne comprends pas ta question, peux tu la reformuler s\'il te plaît?')
                print()
                continue
            if len(line) !=0:
                index= find_right_index(LINE, DICT, méthode)
                if time == None and list_city == None:
                     print('Bot: ', pairs_trains[index][1])
                elif list_city ==None:
                    print('Bot: {}, {}'.format(time_word, pairs_trains[index][1]))
                elif time == None:
                    cities = ' '.join(list_city)
                    print('Bot: À {}, {}'.format(cities, pairs_trains[index][1]))
                else:
                    cities = ' '.join(list_city)
                    print('Bot: À {}, {}, {}'.format(cities, time_word, pairs_trains[index][1]))
               
                LINE = clear_line(line)   
                index = find_right_index(LINE, DICT, METHODE[0])
                #print('YOU: ', line)
                #print('BOT: ', pairs_trains[index][1])
                print()
            

def get_all_and_all_convos():
    convos = []
    file1 = 'data/convos27juin_test.txt'
    file2 = 'data/convos27juin_train.txt'
    files = [file1, file2]
    for file in files:
        with open(file, 'r') as f:
            i=0
            for line in f:
                if i%2==0:
                    question = line.strip()
                    if '++++' in question:
                         question = question[9:]
                else:
                    answer = line.strip()
                    if '++++' in answer:
                        answer = answer[9:]
                    convos.append([question, answer])
                i+=1
        f.close()
    return convos


def get_all_convos_in_movies():
    convos = []
    raw_questions = []
    raw_convos = []
    for root, dirs, files in os.walk("soustitre"):
        for file in files:
            if file.endswith(".txt"):
                path = os.path.join(root, file)
                i = 0
                with open(path, 'r') as f:
                    get_ = False
                    for line in f.readlines():
                        line = line.strip('-')
                        if i > 3:
                            if '?' in line and get_ == False:
                                question = line.strip()
                                get_ = True
                            else:
                                if get_ == True:
                                    answer = line.strip()
                                    convos.append([clear_line(question), answer])
                                    raw_convos.append([question, answer])
                                    raw_questions.append(question)
                                    get_ = False
                          
                        i +=1
    index_of_train = random.sample(range(len(convos)), int(.8*len(convos)))
    raw_questions = [raw_questions[i] for i in index_of_train]
    pairs_trains = [convos[i] for i in index_of_train]
    pairs_tests = [convos[i] for i in range(len(convos)) if i not in index_of_train]
    return pairs_trains, pairs_tests, raw_questions, raw_convos


    
def dict_of_all_score_(data, KEY_LIST, sujet):
    """
    return score of all word in all doc"""
    DICT = {}
    for doc in data:
        DICT[doc]=dict_doc_score_(doc,KEY_LIST, sujet)
    return DICT 



def find_right_index_(new_doc):
    

    index=0
    biggest_score = score_by_new_doc_(new_doc, DATA[0],dictionary)
    for i in range(len(DATA)):
        boolean1 = score_by_new_doc_(new_doc, DATA[i], dictionary) > biggest_score
        boolean2 = score_by_new_doc_(new_doc, DATA[i], dictionary) == biggest_score
        boolean3 = len(DATA[index]) >len(DATA[i])
        if boolean1 or (boolean2 and boolean3):
            biggest_score = score_by_new_doc_(new_doc, DATA[i], dictionary)
            index = i
    if biggest_score == 0:
        index = random.choice([i for i in range(len(DATA))])
    return index



        



def request_missing_infos(new_question, good_question, dict_):
    
    if 'to_return' in good_question:
        return True
    score1 = score_by_new_doc_(new_question, good_question, dict_)
    score2 = 0
    for word in dict_[good_question]:
        score2 += dict_[good_question][word]
    if score1/score2 > 0.6:
        return  True
    else:
        return  False
    
def to_print_answer(line):
    line = str(line)
    first = line[0]
    line = first.upper()+ line[1:]
    if line[-1] not in ['?','.','!']:
        line = line+'.'
    line = line.strip(' ')
    return line
    
def chat_(): 
      
    DICTIONARY =  dict_for_all_doc(DATA)      
    print('Bonjour, le bot d\'Avicen à ton écoute')
    path = 'processed/chat_tfidf.txt'
    f =  open(path, 'a+') 
    while True:
            line = str(input('Vous: '))
            if len(line) < 2:
                break
            time, time_word = _find_right_time(line, COMPLET_DICT_TIME)
            list_city = _find_right_geography(line, CITY_LIST)
            if time_word is not None:
                line = line.replace(time_word, '')
            if list_city is not None:
                for city in list_city:
                    line = line.replace(city, '')
            LINE = clear_line(line)
            unknown_word =0
            for word in LINE.split():
                if word not in LIST_OF_WORDS:
                    unknown_word += 1
            if unknown_word > 1:
                print('Désolé, je ne comprends pas ta question, peux tu la reformuler s\'il te plaît?')                    
                print()
                continue
            if len(line) !=0:
                index= find_right_index_(LINE)
                if time == None and list_city == None:
                     print('Bot: ', convos_train[index][1])
                elif list_city ==None:
                    print('{}, {}'.format(time_word, convos_train[index][1]))
                elif time == None:
                    cities = ' '.join(list_city)
                    print('À {}, {}'.format(cities, convos_train[index][1]))
                else:
                    cities = ' '.join(list_city)
                    print('À {}, {}, {}'.format(cities, time_word, convos_train[index][1]))
                
    
def dict_doc_score_(doc, KEY_LIST, sujet): 
    """
    We return score of all one words and all two consequent words"""

    list_word_ = split_words(doc)    
    return {word: tf_idf_(word, doc, KEY_LIST, sujet) for word in list_word_}
   

    
def score_by_new_doc_(new_doc, doc, DICT):
    
    list_word_in_doc = split_words(new_doc)
    score = 0
    for word in list_word_in_doc:
            if word in DICT[doc]:
                 score += DICT[doc][word]
    return score


def tf_idf_standard(word, doc):
    
    df = 0
    doc_split = split_words(doc)
    if word in doc_split:
        tf = 1
    else:
        tf = 0
    for doc_ in DATA:
        if word in doc_:
            df+=1
    if df >0:
        return tf*math.log(len(DATA)/df, 10)
    else:
        return 0
    
def dict_of_entropy_for_all_terms(terms, K):
    """
    Nous utilisons la méthode donné dans Class Specific TF-IDF Boosting 2018
    K is a tuning parameter of method"""
    dictionnary_terms ={}
    H_max =0
    for term in terms:
        H = 0
        df = 0
        for data2 in DATA:
            if term in data2:
                df+=1
        for data in DATA:
            if term in data:
                tf_idf = - 1/df*math.log(1/df, 2)
            else:
                tf_idf = 0
            H += tf_idf
        if H > H_max:
            H_max = H
        dictionnary_terms[term] = H
    for term in terms:
        dictionnary_terms[term] = (H_max - dictionnary_terms[term])/(H_max*K)
    return dictionnary_terms

def dict_of_all_doc_by_new_method(K):
    
    """
    This will be the dictionnary which is the heart of method"""
    dict_of_new_method = {}
    terms = get_all_terms()
    dictionnary_terms = dict_of_entropy_for_all_terms(terms, K)
    for doc in DATA:
        terms_of_this_doc = split_words(doc)
        dict_term_for_this_doc = {term: tf_idf_standard(term, doc)+dictionnary_terms[term]\
                                  if tf_idf_standard(term, doc) >0 else 0\
                                  for term in terms_of_this_doc}
        dict_of_new_method[doc] = dict_term_for_this_doc
    return dict_of_new_method

def get_all_terms():
    """
    return all terms for training
    """
    all_terms = []
    for data in DATA:
        all_terms.extend(split_words(data))
    return all_terms
        
    
    
def get_best_parameter():
    
    start = time.time()
    K = [1/4000, 1/1000, 1/256, 1/64]
    k_best = 1
    F_best = 0
    for k in K:      
        dictionary =  dict_of_all_doc_by_new_method(k)
        F_score = return_F_score()
        print('k= {}, F score = {}'.format(k, F_score))
        if F_score >F_best:
            F_best = F_score
            k_best = k
    print('best score is {} and best k is {}'.format(F_best, k_best))
    print('running time is ', int(time.time()-start), 's.')
    return k_best, F_best
        
        
        


def data_processed():
    """
    Traiter et retourner les conversations avec les questions bien nettoyées
    les questions sans nettoyer et le couple pour test
    """
    convos1, questions1 = get_all_convos(STOPWORDS,'data/trains_sujet.txt')
    convos2, questions2  = get_all_convos(STOPWORDS, 'data/trains_hors_sujet.txt')
    convos3, questions3 = get_all_convos(STOPWORDS, 'data/convos3septembre_train.txt')
    convos1.extend(convos2)
    convos1.extend(convos3)
    questions1.extend(questions2)
    questions1.extend(questions3)
    convos_test1, questions_test1 = get_all_convos(STOPWORDS, 'data/tests_16juillet.txt')
    convos_test2, questions_test2 = get_all_convos(STOPWORDS, 'data/convos3septembre_test.txt')
    convos_test1.extend(convos_test2)
    questions_test1.extend(questions_test2)
    index_suff = [i for i in range(len(convos1))]
    random.shuffle(index_suff)
    convos, questions = [], []
    for i in index_suff:
        convos.append(convos1[i])
        questions.append(questions1[i])
    index_suff_ = [i for i in range(len(convos_test1))]
    random.shuffle(index_suff_)
    convos_test, questions_test = [], []
    for i in index_suff_:
        convos_test.append(convos_test1[i])
        questions_test.append(questions_test1[i])    
    return convos, questions, convos_test, questions_test


        

def convos_to_questions(convos):
    """
    prend tous les conversations dans corpus et renvoie un array de documents avec
    des vocabulaires séparés"""
    
    data = []
    for convo in convos:
        data.append(convo[0])
    return data

def get_answer(index):
    if index != None:
        return convos_train[index][1]
    else:
        return 'Désolé, je ne comprends pas ta question'
    
def list_words():
    
    list_ = []
    for data in DATA:
        for term in split_words(data):
            if term not in list_:
                list_.append(term)
    return list_

def test_chat():
        
    CONVOS_TRAIN, _, _, _ = data_processed()
    return random.choice(CONVOS_TRAIN)
if '__name__' == '__main__':
    chat_()
        

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
chat_()

Bonjour, le bot d'Avicen à ton écoute
Vous: 
